<a href="https://colab.research.google.com/github/chetools/CVE2005_Spring2025/blob/main/BallTrajectory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
from plotly.subplots import make_subplots
from scipy.integrate import solve_ivp
from scipy.optimize import root

In [4]:
def Cd(Re):
    a=Re/5
    b=Re/2.63e5
    c=Re/1e6
    return 24/Re + 2.6*a/(1+a**1.52) +0.411*b**(-25)/(1+b**(-25)) + 0.25*c/(1+c)

In [5]:
Re = np.logspace(-1,10,200)

fig = make_subplots()
fig.add_scatter(x=Re, y=Cd(Re))
fig.update_xaxes(type="log")
fig.update_yaxes(type="log")
fig.update_layout(width=600,height=400,template='plotly_dark')

In [6]:
# F = rho * Cd * A * V^2 / 2
# Stokes Regime, Cd = 24/Re = 24*mu/(rho * V * D)

# F = rho * 24*mu/(rho * V * D) * pi * D^2 / 4 * V^2 /2
# F = 3 * mu* pi * V* D

D = 1e-7
mu = 1.8e-5
rho = 1.29

rho_particle = 2e3

weight = rho_particle * np.pi/6* D**3 * 9.81


In [7]:
# drop a sphere from an airplane
D = 1.8 #m
weight = 100. #kg
# F = rho * Cd * np.pi*(1.8) * V^2 / 2

In [8]:
Vguess = 100 #mph
Vguess = Vguess * 1610/3600

In [9]:
def net_force(V):
    Re = np.abs(rho * V * D / mu)
    A = (np.pi*D**2) / 4
    return np.copysign((rho * Cd(Re) * A * V**2) / 2, V) - weight

In [10]:
root(net_force, [40.])

 message: The solution converged.
 success: True
  status: 1
     fun: [-1.421e-14]
       x: [ 1.845e+01]
  method: hybr
    nfev: 12
    fjac: [[-1.000e+00]]
       r: [-1.237e+01]
     qtf: [-3.573e-11]

In [32]:
D = 0.22 #m
A = np.pi*(D**2)/4
mu = 1.8e-5
rho = 1.29
m = 0.425 #kg
g= 9.81 #m/s2


def hit_ground(t, vec):
    x,y, vx, vy = vec
    return y

hit_ground.terminal = True
hit_ground.direction = -1

def rhs(t, vec):
    x,y, vx, vy = vec

    V = np.array([vx,vy])
    speed = np.linalg.norm(V)
    Re = rho * speed *D/mu
    Fd = -V/speed * Cd(Re) * A * rho * (speed**2)/2
    Fx, Fy = Fd
    dx = vx
    dy = vy
    dvx = Fx/ m
    dvy = -g + Fy/m

    return [dx, dy, dvx, dvy]

In [33]:
tend = 5.

Vi = 100 # mph
Vi = Vi * 1610. / 3600  #m/s
theta = 30 * np.pi/180 #rad
vxi = Vi * np.cos(theta)
vyi = Vi * np.sin(theta)


res = solve_ivp(rhs, (0,tend), [0,0, vxi, vyi], dense_output=True, events=hit_ground)

In [39]:
tplot = np.linspace(0,res.t_events[0][0], 100)
x,y,vx,vy = res.sol(tplot)

In [40]:
fig = make_subplots()
fig.add_scatter(x=x,y=y, mode='markers')
fig.update_layout(width=600,height=400,template='plotly_dark')